In [1]:
# get OpenNMT
from google.colab import drive
drive.mount('/content/gdrive')
!ls
!git clone -b v1.2.0 https://github.com/OpenNMT/OpenNMT-py.git
!git pull

Mounted at /content/gdrive
data.zip  gdrive  preprocessed_data.zip  sample_data
Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 21369, done.
remote: Counting objects: 100% (2217/2217), done.
remote: Compressing objects: 100% (1015/1015), done.
remote: Total 21369 (delta 1327), reused 1933 (delta 1162), pack-reused 19152
Receiving objects: 100% (21369/21369), 309.30 MiB | 31.72 MiB/s, done.
Resolving deltas: 100% (15243/15243), done.
fatal: not a git repository (or any of the parent directories): .git


In [2]:
!pip install -r ./OpenNMT-py/floyd_requirements.txt

  Cloning https://github.com/pytorch/text to /tmp/pip-req-build-vmw46hv4
  Running command git clone --filter=blob:none --quiet https://github.com/pytorch/text /tmp/pip-req-build-vmw46hv4
  Resolved https://github.com/pytorch/text to commit c0d0685a8a6e4c5a0f05629330c3a2d1bc5596d0
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [3]:
!unzip data.zip
!unzip preprocessed_data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/train.fa           
  inflating: __MACOSX/data/._train.fa  
  inflating: data/train.en           
  inflating: __MACOSX/data/._train.en  
  inflating: data/test.en            
  inflating: __MACOSX/data/._test.en  
  inflating: data/test.fa            
  inflating: __MACOSX/data/._test.fa  
  inflating: data/valid.en           
  inflating: __MACOSX/data/._valid.en  
  inflating: data/valid.fa           
  inflating: __MACOSX/data/._valid.fa  
Archive:  preprocessed_data.zip
   creating: preprocessed_data/
  inflating: __MACOSX/._preprocessed_data  
  inflating: preprocessed_data/train.fa  
  inflating: __MACOSX/preprocessed_data/._train.fa  
  inflating: preprocessed_data/train.en  
  inflating: __MACOSX/preprocessed_data/._train.en  
  inflating: preprocessed_data/test.en  
  inflating: __MACOSX/preprocessed_data/._test.en  
  inflating: preprocessed_data/test.fa  
  inflating: __MACOSX/prep

In [4]:
# BPE
!mkdir BPE
!python OpenNMT-py/tools/learn_bpe.py -i preprocessed_data/train.en -o BPE/src.code -s 10000
!python OpenNMT-py/tools/learn_bpe.py -i preprocessed_data/train.fa -o BPE/tgt.code -s 10000
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/train.en -o BPE/train-bpe.en
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/valid.en -o BPE/valid-bpe.en
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/test.en -o BPE/test-bpe.en.txt
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/train.fa -o BPE/train-bpe.fa
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/valid.fa -o BPE/valid-bpe.fa
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/test.fa -o BPE/test-bpe.fa

In [ ]:
# Preprocess
!pip install torchtext==0.4
!pip install configargparse
!mkdir translate

In [ ]:
!python OpenNMT-py/preprocess.py -train_src BPE/train-bpe.en -train_tgt BPE/train-bpe.fa -valid_src BPE/valid-bpe.en -valid_tgt BPE/valid-bpe.fa -save_data translate/data -overwrite

### train

In [ ]:
# Train
!python OpenNMT-py/train.py -data translate/data -save_model translate/model -train_steps 50000 -save_checkpoint_steps 5000 -valid_steps 1000 -world_size 1 -gpu_rank 0

In [ ]:
# Test
!mkdir pred
!mkdir pred/test
!mkdir pred/validation

In [ ]:
from tqdm import tqdm

bleus = []
for i in tqdm(range(5000,50001,5000)) :
  c1 = !python OpenNMT-py/translate.py -model translate/model_step_{i}.pt -src data/valid.en -output pred/validation/pred_valid_{i}.txt -replace_unk
  c2 = !sed -i "s/@@ //g"  pred/validation/pred_valid_{i}.txt
  c3 = !perl OpenNMT-py/tools/multi-bleu.perl data/valid.fa < pred/validation/pred_valid_{i}.txt
  bleus.append(c3)

In [ ]:
bleus

In [ ]:
!python OpenNMT-py/translate.py -model translate/model_step_50000.pt -src data/test.en -output pred/test/pred_test.txt -replace_unk -verbose

In [ ]:
!sed -i "s/@@ //g"  pred/test/pred_test.txt

In [ ]:
!perl OpenNMT-py/tools/multi-bleu.perl data/test.fa < pred/test/pred_test.txt